In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf


In [4]:
payload = {'alertedParty': {'ALL_PARTY_TYPES': ['Individual', 'Individual'], 'AP_PARTY_TYPE': 'I'}, 'metadata': {'datasetType': 'WM_ADDRESS'}, 'xml_fields': {'PARTY1_ORGANIZATION_NAME': None, 'ORGANIZATIONPERSONIND': None}, 'alertSupplementalInfo': {'supplementalInfo': []}}

In [5]:
from dataclasses import dataclass
from typing import List


@dataclass
class SourceField:
    source: str
    field: str

@dataclass
class Function:
    target_field: str
    source: List[SourceField]
    target_collection: str = None
    method: str = ""
    method_name: str = ""
    pattern_method: str = ""
        
        
    def __post_init__(self, *args, **kwargs):
        method = kwargs.get("method")
        if method:
            method_name = method
            pattern_method = f"_{method}"
    


flow_config = OmegaConf.load(os.path.join("config", "pipeline", "flow.yaml"))
func_map = {}
for column_transformation in flow_config:
    name = str(column_transformation)
    item = dict(flow_config[column_transformation])
    item['source'] = [SourceField(str(field_type), str(field)) for field_type in item['source'] for field in item['source'][field_type]]
    dict_input = dict(**item, **{'target_field': column_transformation})
    func_map.update({name: Function(**dict_input)})
#     print(name)
#     print(f"pattern_set_up_party_type({flow_config[column_transformation]['source']})")
    

In [6]:
func_map['AP_PARTY_TYPE']

Function(target_field='AP_PARTY_TYPE', source=[SourceField(source='alertedParty', field='ALL_PARTY_TYPES'), SourceField(source='metadata', field='datasetType'), SourceField(source='xml_fields', field='PARTY1_ORGANIZATION_NAME'), SourceField(source='xml_fields', field='ORGANIZATIONPERSONIND'), SourceField(source='alertSupplementalInfo', field='supplementalInfo.legalFormName')], target_collection='alertedParty', method='set_up_party_type', method_name='', pattern_method='')

In [38]:
def draw_design(func_name, field="", nested_level=0):
    tabs = nested_level * '\t'
    design = f"{func_name}"
    
    
    try:
        func = func_map[func_name]
        design += f"\n{tabs}- value: {payload[func.target_collection][func_name]}"
        if func.source:
            design += f"\n{tabs}- collection: {func.target_collection}"
        if func.method or func.method_name:
            design += f"\n{tabs}- created by: {func.method if func.method else func.method_name}"
        
    except KeyError:
        try:
            design += f"\n{tabs}- value: {payload[field][func_name]}"
        except:
            value = None
            if func_name == "partyType":
                value = "Individual"
            design += f"\n{tabs}- value: {value}"
        if field:
            design += f"\n{tabs}- collection: {field}"
        if func_name == "partyType":
            f_tabs = (nested_level-1) * '\t'
            design += f"\n{f_tabs+(len('<-')+2)* ' '}" + design
        return design
    design +=  f"\n{tabs}- source fields:"
    for field in func.source:
        design += f"\n{tabs}  <-" + draw_design(field.field, field.source, nested_level+1) + f"\n{tabs}"
    return design.replace("\n\n", "\n")

In [39]:
print(draw_design('AP_PARTY_TYPE'))

AP_PARTY_TYPE
- value: I
- collection: alertedParty
- created by: set_up_party_type
- source fields:
  <-ALL_PARTY_TYPES
	- value: ['Individual', 'Individual']
	- collection: alertedParty
	- created by: collect_party_types
	- source fields:
	  <-partyType
		- value: Individual
		- collection: parties
	    partyType
		- value: Individual
		- collection: parties
	
  <-datasetType
	- value: WM_ADDRESS
	- collection: metadata
  <-PARTY1_ORGANIZATION_NAME
	- value: None
	- collection: xml_fields
  <-ORGANIZATIONPERSONIND
	- value: None
	- collection: xml_fields
  <-supplementalInfo.legalFormName
	- value: None
	- collection: alertSupplementalInfo

